In [8]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from sklearn.linear_model import Lasso
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error
from pysindy.utils.odes import lorenz
from pynew import *

# Ignore matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Seed the random number generators for reproducibility
np.random.seed(100)

# integration keywords for solve_ivp, typically needed for chaotic systems
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

In [9]:
# Generate measurement data
dt = 0.001
t_train = np.arange(0, 10, dt)
t_train_span = (t_train[0], t_train[-1])
u0_train = [-8, 8, 27]
u_train = solve_ivp(lorenz, t_train_span, u0_train, 
                    t_eval=t_train, **integrator_keywords).y.T
import random as rand
error_scale = 0.1
noise = np.random.randn(10000, 3) * error_scale
u_train = u_train + noise

# Instantiate and fit the SINDy model with u_dot
u_dot = FiniteDifference()._differentiate(u_train, t=dt)
model = SINDy()
model.fit(u_train, x_dot=u_dot, t=dt)
model.print()

# Define weak form ODE library
# defaults to derivative_order = 0 if not specified,
# and if spatial_grid is not specified, defaults to None,
# which allows weak form ODEs.
library_functions = [lambda x: x, lambda x: x * x, lambda x, y: x * y]
library_function_names = [lambda x: x, lambda x: x + x, lambda x, y: x + y]
ode_lib = WeakPDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    spatiotemporal_grid=t_train,
    is_uniform=True,
    K=100,
)

# Instantiate and fit the SINDy model with the integral of u_dot
optimizer = SR3(
    threshold=0.05, 
    thresholder="l1", 
    max_iter=1000, 
    normalize_columns=True, 
    tol=1e-1
)
model = SINDy(feature_library=ode_lib, optimizer=optimizer)
model.fit(u_train)
model.print()

(x0)' = -10.022 x0 + 10.017 x1
(x1)' = 27.850 x0 + -0.949 x1 + -0.997 x0 x2
(x2)' = -2.668 x2 + 1.000 x0 x1
This is the right things
(x0)' = -9.997 x0 + 9.994 x1
(x1)' = 27.804 x0 + -0.870 x1 + 0.001 x0x0 + -0.994 x0x2 + -0.004 x1x2
(x2)' = -2.670 x2 + 0.001 x1x1 + 0.999 x0x1
